# Simple CNN for Cell Segmentation
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MateiGoidan/CellSeg/blob/main/notebooks/01_simple_cnn.ipynb)

---
## 1. Setup
Import dependencies and configure environment.


In [ ]:
# @title Conectare la drive
from google.colab import drive

drive.mount('/content/drive/')

In [ ]:
# @title Imports

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import tensorflow.keras.backend as K
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt

# Dataset

In [ ]:
# @title Setari cale

train_img_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/train/img/cls"
train_bin_mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/train/bin_mask/cls"
train_mult_mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/train/mult_mask/cls"

validation_img_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/validation/img/cls"
validation_bin_mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/validation/bin_mask/cls"
validation_mult_mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/validation/mult_mask/cls"


In [ ]:
# @title Incarcam path-urile

def load_paths(imd_dir, mask_dir):
  image_paths = sorted([os.path.join(imd_dir, file_name) for file_name in os.listdir(imd_dir) if file_name.endswith(".tif")])
  mask_paths = sorted([os.path.join(mask_dir, file_name) for file_name in os.listdir(mask_dir) if file_name.endswith(".tif")])
  return image_paths, mask_paths

train_img_paths, train_mask_paths = load_paths(train_img_path, train_mult_mask_path)

validation_img_paths, validation_mask_paths = load_paths(validation_img_path, validation_mult_mask_path)

In [ ]:
# @title Augumentare data

def augment_data(image, mask):
  # Random flip orizontal
  if tf.random.uniform(()) > 0.5:
    image = tf.image.flip_left_right(image)
    mask = tf.image.flip_left_right(mask)

  # Random flip vertical
  if tf.random.uniform(()) > 0.5:
    image = tf.image.flip_up_down(image)
    mask = tf.image.flip_up_down(mask)

  # Random rotire
  angle = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
  image = tf.image.rot90(image, angle)
  mask = tf.image.rot90(mask, angle)

  # Random zoom
  if tf.random.uniform(()) > 0.5:
    scales = list(np.arange(0.8, 1.0, 0.01))
    scale = np.random.choice(scales)
    new_height = tf.cast(1040 * scale, tf.int32)
    new_width = tf.cast(1392 * scale, tf.int32)

    image = tf.image.resize(image, (new_height, new_width))
    mask = tf.image.resize(mask, (new_height, new_width), method='nearest')

    image = tf.image.resize_with_crop_or_pad(image, 1040, 1392)
    mask = tf.image.resize_with_crop_or_pad(mask, 1040, 1392)

  return image, mask

In [ ]:
# @title Preprocesare imagine + masca

def load_tif_with_pil(path):
  img = Image.open(path.numpy().decode('utf-8'))
  img = img.convert('L') # Convertim la grayscale
  img = np.array(img, dtype=np.float32)
  return img

def preprocess_bin(image_path, mask_path):
  # Citim imaginea
  image = tf.py_function(func=load_tif_with_pil, inp=[image_path], Tout=tf.float32)
  image = tf.expand_dims(image, axis=-1)
  image.set_shape([1040, 1392, 1])

  # Citim masca
  mask = tf.py_function(func=load_tif_with_pil, inp=[image_path], Tout=tf.float32)
  mask = tf.expand_dims(mask, axis=-1)
  mask.set_shape([1040, 1392, 1])

  # Normalizare si
  image = tf.cast(image, tf.float32) / 255.0
  mask = tf.cast(mask, tf.float32) / 255.0

  # image.set_shape([1040, 1392, 1])
  # mask.set_shape([1040, 1392, 1])

  return image, mask

def preprocess_mult(image_path, mask_path):
  # Citim imaginea
  image = tf.py_function(func=load_tif_with_pil, inp=[image_path], Tout=tf.float32)
  image = tf.expand_dims(image, axis=-1)
  image.set_shape([1040, 1392, 1])

  # Citim masca
  mask = tf.py_function(func=load_tif_with_pil, inp=[mask_path], Tout=tf.float32)
  mask = tf.expand_dims(mask, axis=-1)
  mask.set_shape([1040, 1392, 1])

  # Normalizare
  image = tf.cast(image, tf.float32) / 255.0
  mask = tf.cast(mask, tf.float32)

  return image, mask

In [ ]:
# @title Creare dataset-ului

def create_dataset(image_paths, mask_paths, batch_size=2, shuffle=True, augment=False):
  dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
  dataset = dataset.map(preprocess_mult, num_parallel_calls=tf.data.AUTOTUNE)

  if augment:
    dataset = dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)

  if shuffle:
    dataset = dataset.shuffle(buffer_size=100)

  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(tf.data.AUTOTUNE)
  return dataset

train_dataset = create_dataset(train_img_paths, train_mask_paths, augment=True)
validation_dataset = create_dataset(validation_img_paths, validation_mask_paths)

In [ ]:
# @title Verificare shape imagini + masca

for images, masks in validation_dataset.take(1):
  print("Shape imagini batch:", images.shape)
  print("Shape măști batch:", masks.shape)


# Model CNN Simplu Binar

In [ ]:
# @title Model CNN simplu

from tensorflow.keras import layers, models

def build_simple_cnn(input_shape=(1040, 1392, 1)):
  input = layers.Input(shape=input_shape)

  # Encoder
  x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input)
  x = layers.MaxPooling2D((2, 2), padding='same')(x)

  x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = layers.MaxPooling2D((2, 2), padding='same')(x)

  # Decoder
  x = layers.Conv2DTranspose(32, (2, 2), strides=2, activation='relu', padding='same')(x)
  x = layers.Conv2DTranspose(16, (2, 2), strides=2, activation='relu', padding='same')(x)

  # Strat de iesire
  outputs = layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)

  model = models.Model(inputs=input, outputs=outputs)
  return model

In [ ]:
# @title Metrici custom

import tensorflow.keras.backend as K

# Intersection over Union (IoU)
def iou(y_true, y_pred):
  y_pred = K.cast(y_pred > 0.5, 'float32')
  intersection = K.sum(y_true * y_pred)
  union = K.sum(y_true) + K.sum(y_pred) - intersection
  return intersection / (union + K.epsilon())

def dice(y_true, y_pred):
  y_pred = K.cast(y_pred > 0.5, 'float32')
  intersection = K.sum(y_true * y_pred)
  dice = (2. * intersection) / (K.sum(y_true) + K.sum(y_pred) + K.epsilon())
  return dice

In [ ]:
# @title Construire si compilare a modelului

simple_CNN = build_simple_cnn()

simple_CNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', iou, dice])

simple_CNN.summary()

In [ ]:
# @title Antranare model

train = simple_CNN.fit(train_dataset, validation_data=validation_dataset, epochs=5)

In [ ]:
for images, masks in validation_dataset.take(1):
  print("Pixelii unici în ground truth:", tf.unique(tf.reshape(masks, [-1]))[0].numpy())

In [ ]:
from PIL import Image
import numpy as np

# Exemplu: deschidem un mask .tif brut
mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/train/bin_mask/cls/0001.tif"  # înlocuiește cu un fișier real
mask_image = Image.open(mask_path)
mask_array = np.array(mask_image)

print("Valori unice în mască:", np.unique(mask_array))


In [ ]:
# import matplotlib.pyplot as plt

# for images, masks in validation_dataset.take(1):
#     preds = simple_CNN.predict(images)
#     preds = (preds > 0.5).astype(np.float32)  # threshold la 0.5

#     plt.figure(figsize=(6, 20))
#     for i in range(2):  # 2 imagini
#         plt.subplot(3, 2, 1+i)
#         plt.imshow(tf.squeeze(images[i]), cmap='gray')
#         plt.title('Imagine originală')

#         plt.subplot(3, 2, 3+i)
#         plt.imshow(tf.squeeze(masks[i]), cmap='gray')
#         plt.title('Ground truth (etichete)')

#         plt.subplot(3, 2, 5+i)
#         plt.imshow(tf.squeeze(preds[i]), cmap='gray')
#         plt.title('Predicție model')
#     plt.show()

simple_CNN.evaluate(validation_dataset)

# Model CNN Simplu Multi-Clasa

In [ ]:
# @title Model Multi-Clasa

from tensorflow.keras import layers, models

def build_simple_cnn_multiclass(input_shape=(1040, 1392, 1), num_classes=8):
  input = layers.Input(shape=input_shape)

  # Encoder
  x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input)
  x = layers.MaxPooling2D((2, 2))(x)

  x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = layers.MaxPooling2D((2, 2))(x)

  # Decoder
  x = layers.Conv2DTranspose(32, (2, 2), strides=2, activation='relu', padding='same')(x)
  x = layers.Conv2DTranspose(16, (2, 2), strides=2, activation='relu', padding='same')(x)

  # Output
  outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(x)

  model = models.Model(inputs=input, outputs=outputs)
  return model

In [ ]:
# @title Metrici personalizati

import tensorflow.keras.backend as K

def iou_multiclass(y_true, y_pred):
  # Convertim predictia softmax in argmax
  y_pred = tf.argmax(y_pred, axis=-1, output_type=tf.int32)

  y_true = tf.squeeze(y_true, axis=-1) # Daca masca are inca un canal 1
  y_true = tf.cast(y_true, tf.int32)

  intersection = tf.reduce_sum(tf.cast(tf.equal(y_true, y_pred), tf.float32))
  union = tf.reduce_sum(tf.ones_like(y_true, dtype=tf.float32))

  return intersection / (union + K.epsilon())

def dice_multiclass(y_true, y_pred):
  # Convertim predictia softmax in argmax
  y_pred = tf.argmax(y_pred, axis=-1, output_type=tf.int32)

  y_true = tf.squeeze(y_true, axis=-1) # Daca masca are inca un canal 1
  y_true = tf.cast(y_true, tf.int32)

  y_true_one_hot = tf.one_hot(y_true, depth=8)
  y_pred_one_hot = tf.one_hot(y_pred, depth=8)

  intersection = tf.reduce_sum(y_true_one_hot * y_pred_one_hot)
  denominator = tf.reduce_sum(y_true_one_hot) + tf.reduce_sum(y_pred_one_hot)

  return 2.0 * intersection / (denominator + K.epsilon())

In [ ]:
# @title Compilam modelul

multiclass_CNN = build_simple_cnn_multiclass()

multiclass_CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=["sparse_categorical_accuracy", iou_multiclass, dice_multiclass])

multiclass_CNN.summary()

In [ ]:
# @title Antrenare

train = multiclass_CNN.fit(train_dataset, validation_data=validation_dataset, epochs=5)

In [ ]:
# @title Testam modelul

test_img_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/test/img/cls"
test_mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/test/mult_mask/cls"

test_imgs, test_masks = load_paths(test_img_path, test_mask_path)

test_dataset = create_dataset(test_imgs, test_masks, shuffle=False)

results = multiclass_CNN.evaluate(test_dataset)
print("Test loss:", results[0])
print("Test accuracy:", results[1])
print("Test IoU:", results[2])
print("Test Dice:", results[3])

# Vizualizare

In [ ]:
# @title Colorare masca

palette = np.array([
  [0, 0, 0],       # 0: Fundal - Negru
  [255, 0, 0],     # 1: Rod (bacterie) - Rosu
  [0, 255, 0],     # 2: RBC/WBC - Verde
  [0, 0, 255],     # 3: Yeast - Albastru
  [255, 255, 0],   # 4: Miscellaneous - Galben
  [255, 0, 255],   # 5: Single EPC - Magenta
  [0, 255, 255],   # 6: Small EPC sheet - Cyan
  [255, 255, 255]  # 7: Large EPC sheet - Alb
], dtype=np.uint8)

In [ ]:
# @ Vizualizare imagine + ground truth + predictie

import os

# 1. Creăm directorul pentru salvare dacă nu există
output_dir = "/content/outputs_test"
os.makedirs(output_dir, exist_ok=True)

# 2. Funcție color map (o refacem puțin robustă)
def apply_color_map(mask):
    mask = tf.squeeze(mask, axis=-1)
    mask = tf.cast(mask, tf.int32)
    mask = mask.numpy()
    color_mask = palette[mask]
    return color_mask

# 3. Parcurgem batch-uri și salvăm imaginile
for batch_idx, (images, masks) in enumerate(test_dataset.take(5)):  # luam 5 batch-uri
    preds = multiclass_CNN.predict(images)
    preds = tf.argmax(preds, axis=-1)
    preds = tf.expand_dims(preds, axis=-1)

    batch_size = images.shape[0]

    for i in range(batch_size):
        plt.figure(figsize=(24, 8))

        # Imagine originală
        plt.subplot(1, 3, 1)
        plt.imshow(tf.squeeze(images[i]), cmap='gray')
        plt.title('Imagine originală')
        plt.axis('off')

        # Ground Truth colorat
        plt.subplot(1, 3, 2)
        gt_color = apply_color_map(masks[i])
        plt.imshow(gt_color)
        plt.title('Ground Truth')
        plt.axis('off')

        # Predicție multiclass_CNN colorată
        plt.subplot(1, 3, 3)
        pred_color = apply_color_map(preds[i])
        plt.imshow(pred_color)
        plt.title('Predicție multiclass_CNN')
        plt.axis('off')

        plt.tight_layout()

        # Salvăm imaginea
        save_path = os.path.join(output_dir, f"batch{batch_idx}_img{i}.png")
        plt.savefig(save_path)
        plt.close()  # închidem figura ca să nu consume memorie

print(f"Salvat imaginile în {output_dir}")


In [ ]:
for images, masks in train_dataset.take(1):
  print("Shape imagini:", images.shape)
  print("Shape măști:", masks.shape)
  print("Valori unice în măști:", tf.unique(tf.reshape(masks, [-1]))[0].numpy())

  print(multiclass_CNN.summary())
  print(multiclass_CNN.loss)
  print(multiclass_CNN.metrics_names)

  preds = multiclass_CNN.predict(images)
  preds = tf.argmax(preds, axis=-1)
